In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import math
import tensorflow as tf
from scipy.sparse import csr_matrix
from fastpivot import pivot_table, pivot_sparse

In [2]:
movies = pd.read_csv('../datasets/movielens_original/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv('../datasets/movielens_original/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
df = ratings.merge(movies, how = 'inner', on='movieId',)
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [5]:
model = tf.keras.models.load_model('saved_model.h5')

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-21 12:11:59.131292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-21 12:11:59.131653: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
movies_ids = list(set(list(df.movieId.unique())))
users_ids = list(set(list(df.userId.unique())))

dict_movies = {}
index = 0

for ids in sorted(movies_ids):
    dict_movies[ids] = index
    index += 1

dict_users = {}
index = 0
for ids in sorted(users_ids):
    dict_users[ids] = index
    index += 1


In [7]:
user_id = 585
predicted_rating = model.predict([pd.Series([user_id]*len(dict_movies)), pd.Series(dict_movies.values())])
predicted_rating = pd.DataFrame(predicted_rating)
predicted_rating

  21/1846 [..............................] - ETA: 4s  

2023-03-21 12:11:59.689542: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-21 12:11:59.758757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1846/1846 [==============================] - 5s 2ms/step


,0
0,2.932035
1,3.444300
2,3.228229
3,3.583076
4,3.183743
...,...
59042,1.412435
59043,3.443746
59044,4.590337
59045,3.024210


WIlson

In [17]:
user = df[df.userId == 585]
user = csr_matrix((user['rating'].astype(int), (user['userId'].astype(int)-1, user['movieId'].astype(int)-1)))

In [21]:
correlation = df.T.corr()
correlation.head()

/var/folders/fm/r45pd3w141dg56rt5cl8mht80000gn/T/ipykernel_2086/1174692981.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.T.corr()


""


In [22]:
correlation

""


In [18]:
user_ratings = ratings[ratings['userId'] == user_id]

user_movie_ratings = csr_matrix((df['rating'].astype(int), (df['userId'].astype(int)-1, df['movieId'].astype(int)-1)))
# user_movie_ratings.T.corr()

cosine_similarities = cosine_similarity(user_movie_ratings.T, user.T)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 162541 while Y.shape[1] == 585

In [3]:
# create sparse matrix from DataFrame
matrix_sparse = csr_matrix((df['rating'].astype(int), (df['userId'].astype(int), df['movieId'].astype(int))))

# calculate row means
row_means = matrix_sparse.mean(axis=1)

# create sparse matrix of row means
row_means_matrix = csr_matrix(row_means)

# normalize matrix by subtracting row means
matrix_norm = matrix_sparse - row_means_matrix

# calculate cosine similarity between users
user_similarity_cosine = cosine_similarity(matrix_norm.T.fillna(0))

NameError: name 'df' is not defined

In [8]:
pivot_fast

pivot_col,col0,col1,col10,col100,col101,col102,col103,col104,col105,col106,...,col990,col991,col992,col993,col994,col995,col996,col997,col998,col999
pivot_idx,,,,,,,,,,,,,,,,,,,,,
idx0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx10,0.0,-0.336366,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx100,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx101,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,2.363482,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
idx995,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,...,0.000000,1.424630,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
idx996,0.0,0.000000,0.000000,0.0,0.0,0.0,0.152594,0.000000,0.00000,2.597955,...,0.000000,0.000000,0.000000,0.0,-0.437121,1.09762,0.000000,0.0,0.0,0.0
idx997,0.0,1.087749,0.000000,0.0,0.0,0.0,0.000000,0.082645,-0.29158,0.000000,...,0.000000,0.525875,0.128258,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0
